In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
torch.manual_seed(0)
np.random.seed(0)

device = "cuda" if torch.cuda.is_available() else "cpu"

## Define Network

In [12]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc_1 = nn.Linear(28*28*1, 64)
        self.fc_2 = nn.Linear(64, 128)
        self.fc_3 = nn.Linear(128, 10)

    def forward(self, x):
        out = x.view(x.size(0), -1)
        out = F.relu(self.fc_1(out))
        out = F.relu(self.fc_2(out))
        out = self.fc_3(out)

        return out

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1, stride=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.maxpool2 = nn.MaxPool2d(2)
        self.linear1 = nn.Linear(7 * 7 * 64, 200)
        self.relu3 = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(200, 10)

    def forward(self, x):
        out = self.maxpool1(self.relu1(self.conv1(x)))
        out = self.maxpool2(self.relu2(self.conv2(out)))
        out = out.view(out.size(0), -1)
        out = self.relu3(self.linear1(out))
        out = self.linear2(out)
        return out

In [13]:
model = MLP()
model.to(device)

dataset = torch.load("../model_robustness/data/MNIST/dataset.pt")

trainset = dataset["trainset"]
testset = dataset["testset"]
valset = dataset["valset"]

trainloader = DataLoader(
    dataset=trainset,
    batch_size=5,
    shuffle=True,
)
testloader = DataLoader(
    dataset=testset,
    batch_size=5,
    shuffle=False
)
valloader = DataLoader(
    dataset=valset,
    batch_size=5,
    shuffle=False
)

In [21]:
def epoch(mode, device, net, dataloader, optimizer, criterion):
    loss_avg, acc_avg, num_exp = 0,0,0
    if mode == "train":
        net.train()

    else:
        net.eval()

    for i, data in enumerate(dataloader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)

        n_b = labels.shape[0]

        outputs = net(imgs)
        loss = criterion(outputs, labels)

        acc = np.sum(np.equal(np.argmax(outputs.cpu().data.numpy(), axis=-1), labels.cpu().data.numpy()))

        loss_avg += loss.item()
        acc_avg += acc
        num_exp += n_b

        if mode == "train":
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg

## Define perturbed testloader

In [15]:
aux_loader = DataLoader(
    dataset=testset,
    batch_size=len(testset),
    shuffle=False
)

for cln_data, true_label in aux_loader:
    break

cln_data = cln_data.to(device)
true_label = true_label.to(device)

In [16]:
from advertorch.attacks import GradientSignAttack, LinfPGDAttack

pgd_adversary = LinfPGDAttack(
    model,
    loss_fn=nn.CrossEntropyLoss(reduction="sum"),
    eps=0.15,
    nb_iter=40,
    eps_iter=0.01,
    rand_init=True,
    clip_min=0.0,
    clip_max=0.0,
    targeted=False
)

fgsm_adversary = GradientSignAttack(
    model,
    loss_fn=nn.CrossEntropyLoss(reduction="sum"),
    eps=0.15,
    targeted=False
)

In [17]:
# pgd_untargeted = pgd_adversary.perturb(cln_data, true_label)
# fgsm_untargeted = fgsm_adversary.perturb(cln_data, true_label)

target = torch.ones_like(true_label) * 4
pgd_adversary.targeted = True
fgsm_adversary.targeted = True
pgd_targeted = pgd_adversary.perturb(cln_data, target)
fgsm_targeted = pgd_adversary.perturb(cln_data, target)

In [18]:
pgd_data = torch.utils.data.TensorDataset(pgd_targeted, true_label)
pgd_loader = DataLoader(
    dataset=pgd_data,
    batch_size=16,
    shuffle=False
)

fgsm_data = torch.utils.data.TensorDataset(fgsm_targeted, true_label)
fgsm_loader = DataLoader(
    dataset=fgsm_data,
    batch_size=16,
    shuffle=False
)

## Setting up training

In [19]:
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [22]:
for e in range(10):
    train_loss, train_acc = epoch("train", device, model, trainloader, optimizer, criterion)
    print(f"[{e +1}] TRAINING \n loss: {train_loss:.3f}, accuracy: {train_acc:.3f}")

    test_loss, test_acc = epoch("test", device, model, testloader, optimizer, criterion)
    print(f"[{e + 1}] TESTING \n loss: {test_loss:.3f}, accuracy: {test_acc:.3f}")

    test_loss, test_acc = epoch("test", device, model, fgsm_loader, optimizer, criterion)
    print(f"[{e + 1}] PERTURBATION \n loss: {test_loss:.3f}, accuracy: {test_acc:.3f}")

[1] TRAINING 
 loss: 0.548, accuracy: 0.819
[1] TESTING 
 loss: 0.627, accuracy: 0.803
[1] PERTURBATION 
 loss: 4.205, accuracy: 0.089
[2] TRAINING 
 loss: 0.552, accuracy: 0.817
[2] TESTING 
 loss: 0.561, accuracy: 0.825
[2] PERTURBATION 
 loss: 3.731, accuracy: 0.097


KeyboardInterrupt: 

In [10]:
for e in range(10):
    train_loss, train_acc = epoch("train", device, model, trainloader, optimizer, criterion)
    print(f"[{e +1}] TRAINING \n loss: {train_loss:.3f}, accuracy: {train_acc:.3f}")

    test_loss, test_acc = epoch("test", device, model, testloader, optimizer, criterion)
    print(f"[{e + 1}] TESTING \n loss: {test_loss:.3f}, accuracy: {test_acc:.3f}")

    test_loss, test_acc = epoch("test", device, model, pgd_loader, optimizer, criterion)
    print(f"[{e + 1}] PERTURBATION \n loss: {test_loss:.3f}, accuracy: {test_acc:.3f}")

[1] TRAINING 
 loss: 0.130, accuracy: 0.825
[1] TESTING 
 loss: 0.114, accuracy: 0.844
[1] PERTURBATION 
 loss: 0.237, accuracy: 0.089
[2] TRAINING 
 loss: 0.111, accuracy: 0.856
[2] TESTING 
 loss: 0.097, accuracy: 0.881
[2] PERTURBATION 
 loss: 0.247, accuracy: 0.089
[3] TRAINING 
 loss: 0.107, accuracy: 0.863
[3] TESTING 
 loss: 0.108, accuracy: 0.875
[3] PERTURBATION 
 loss: 0.233, accuracy: 0.089
[4] TRAINING 
 loss: 0.104, accuracy: 0.866
[4] TESTING 
 loss: 0.102, accuracy: 0.868
[4] PERTURBATION 
 loss: 0.260, accuracy: 0.089
[5] TRAINING 
 loss: 0.110, accuracy: 0.862
[5] TESTING 
 loss: 0.098, accuracy: 0.866
[5] PERTURBATION 
 loss: 0.246, accuracy: 0.089
[6] TRAINING 
 loss: 0.105, accuracy: 0.857
[6] TESTING 
 loss: 0.104, accuracy: 0.850
[6] PERTURBATION 
 loss: 0.247, accuracy: 0.089
[7] TRAINING 
 loss: 0.106, accuracy: 0.852
[7] TESTING 
 loss: 0.100, accuracy: 0.869
[7] PERTURBATION 
 loss: 0.229, accuracy: 0.089
[8] TRAINING 
 loss: 0.109, accuracy: 0.850
[8] TESTING